#FaceSwap初期設定

In [6]:
!apt-get -y update
!apt-get -y install python3-tk
%cd /notebooks
!git clone https://github.com/YutaAkiyama/faceswap.git
%cd faceswap
!pip install -r requirements.txt

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:2 https://deb.nodesource.com/node_16.x focal InRelease                     
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease                         
Hit:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease                 
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Reading package lists... Done
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-tk is already the newest version (3.8.10-0ubuntu1~20.04).
0 upgraded, 0 newly installed, 0 to remove an

#モデルのダウンロード

In [7]:
!wget -P gfpgan/weights https://huggingface.co/yutaakiyama/inswapper_128.onnx/resolve/main/detection_Resnet50_Final.pth --content-disposition --continue
!wget -P gfpgan/weights https://huggingface.co/yutaakiyama/inswapper_128.onnx/resolve/main/parsing_parsenet.pth --content-disposition --continue
!wget -P models https://huggingface.co/yutaakiyama/inswapper_128.onnx/resolve/main/inswapper_128.onnx --content-disposition --continue
!wget -P models https://huggingface.co/yutaakiyama/inswapper_128.onnx/resolve/main/GFPGANv1.4.pth --content-disposition --continue

--2024-01-11 15:45:30--  https://huggingface.co/yutaakiyama/inswapper_128.onnx/resolve/main/detection_Resnet50_Final.pth
Resolving huggingface.co (huggingface.co)... 18.238.49.117, 18.238.49.10, 18.238.49.70, ...
Connecting to huggingface.co (huggingface.co)|18.238.49.117|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/75/f4/75f48ae56a41f428fddfe65487b97dbe1ad8436c8c4132ce4b5548751f2a07da/6d1de9c2944f2ccddca5f5e010ea5ae64a39845a86311af6fdf30841b0a5a16d?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27detection_Resnet50_Final.pth%3B+filename%3D%22detection_Resnet50_Final.pth%22%3B&Expires=1705247130&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNTI0NzEzMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzc1L2Y0Lzc1ZjQ4YWU1NmE0MWY0MjhmZGRmZTY1NDg3Yjk3ZGJlMWFkODQzNmM4YzQxMzJjZTRiNTU0ODc1MWYyYTA3ZGEvNmQxZGU5YzI5NDRmMmNjZGRjYTVmNWUwM

#顔交換処理

In [1]:
import datetime
import os
import ipywidgets as widgets
from IPython.display import display

# 特定のフォルダからファイルリストを取得する関数
def get_file_list(directory):
    return [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

# ドロップダウンリストを作成する関数
def create_dropdown(directory, description):
    file_list = get_file_list(directory)
    dropdown = widgets.Dropdown(options=file_list, description=description)
    return dropdown

# ファイル選択のためのドロップダウンリストを表示
# ラベルとドロップダウンリストを別々の行に配置するためのコンテナ
target_label = widgets.Label("交換対象の素材 ※Targetフォルダ:")
source_label = widgets.Label("交換したい顔 ※Sourceフォルダ:")
target_dropdown = create_dropdown('/notebooks/faceswap/inputs/target/', '')
source_dropdown = create_dropdown('/notebooks/faceswap/inputs/source/', '')

# 縦に配置するためにVBoxコンテナを使用
dropdown_container = widgets.VBox([target_label, target_dropdown, source_label, source_dropdown])
display(dropdown_container)

# 処理を開始するためのボタンを作成
execute_button = widgets.Button(description='処理開始')
output = widgets.Output()
display(execute_button, output)

def on_execute_clicked(b):
    with output:
        # 選択されたファイル名を取得
        target = target_dropdown.value
        source = source_dropdown.value

        # ファイルパスを構築
        current_time = datetime.datetime.now()
        file_name = current_time.strftime("%Y%m%d%H%M%S")
        INPUT_PATH = f"/notebooks/faceswap/inputs/target/{target}"
        SOURCE_PATH = f"/notebooks/faceswap/inputs/source/{source}"
        extension = ".mp4" if INPUT_PATH.lower().endswith((".mp4", ".mkv")) else ".png"
        OUTPUT_PATH = f"outputs/{file_name}{extension}"

        # 外部コマンドを実行
        print("処理を開始します。しばらくお待ちください...")
        print(f"顔交換が完了したら、{OUTPUT_PATH}に出力されます。")
        !python run.py --target $INPUT_PATH --source $SOURCE_PATH -o $OUTPUT_PATH --execution-provider cuda --frame-processor face_swapper face_enhancer

# ボタンのクリックイベントに関数をバインド
execute_button.on_click(on_execute_clicked)


Button(description='処理開始', style=ButtonStyle())

Output()